In [6]:
!pip install -U tensorflow

In [7]:
import librosa
import numpy as np
import tensorflow as tf
from tensorflow import keras

TypeError: 'type' object is not subscriptable

In [ ]:
import numpy as np
import librosa
import librosa.display
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Dropout
import matplotlib.pyplot as plt

def prepare_data(audio_file):
    # Load the recorded speech signal
    y, sr = librosa.load(audio_file, sr=None)

    # Preprocessing
    # Short-Time Fourier Transform (STFT)
    D = np.abs(librosa.stft(y))

    # Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Prepare input data for the LSTM model
    X = mfccs.T  # Transpose the MFCC matrix to match the shape expected by the LSTM model
    X = X.reshape(X.shape[0], X.shape[1], 1)  # Add channel dimension

    return X

def build_lstm_model(input_shape, num_classes):
    model = Sequential([
        Bidirectional(LSTM(units=128, return_sequences=True), input_shape=input_shape),
        TimeDistributed(Dense(units=64, activation='relu')),
        Dropout(0.2),
        Bidirectional(LSTM(units=64, return_sequences=True)),
        TimeDistributed(Dense(units=num_classes, activation='softmax'))
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def plot_signals(audio_file):
    # Load the recorded speech signal
    y, sr = librosa.load(audio_file, sr=None)

    # Plot the waveform
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.plot(np.arange(len(y)) / sr, y)
    plt.title('Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    # Plot the spectrogram
    plt.subplot(2, 1, 2)
    D = np.abs(librosa.stft(y))
    librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')

    plt.tight_layout()
    plt.show()

# Define the audio file path
audio_file = "speech processing.wav"

# Plot the signals
plot_signals(audio_file)

# Prepare data for LSTM model
X = prepare_data(audio_file)

# Define the number of classes (replace with actual number)
num_classes = 10

# Build LSTM model
model = build_lstm_model(input_shape=X.shape[1:], num_classes=num_classes)

# Display model summary
model.summary()

In [ ]:
!pip install SpeechRecognition



In [ ]:
!pip install pyttsx3

In [ ]:
import os
import tempfile
import speech_recognition as sr
import pyttsx3
import matplotlib.pyplot as plt
import numpy as np
import librosa

# Get a temporary directory where you have write permissions
custom_temp_dir =  r"C:\Users\pavan\OneDrive\Desktop\lab-8"

# Function to segment phonemes for a target word in transcription
def segment_phonemes(transcription, target_word):
    words = transcription.split()
    for word in words:
        if word.lower() == target_word.lower():
            return word

# Function to synthesize speech for a word using segmented phonemes
def synthesize_word(phonemes, output_file):
    engine = pyttsx3.init()
    engine.save_to_file(phonemes, output_file)
    engine.runAndWait()

# Transcribe the audio file
audio_file = "Bhanumathi_weds_Rajat.wav"
recognizer = sr.Recognizer()
with sr.AudioFile(audio_file) as source:
    audio_data = recognizer.record(source)
transcription = recognizer.recognize_google(audio_data)

# Segment phonemes for the word "Bharat"
phonemes = segment_phonemes(transcription, "bharat")

# Synthesize speech for the word "Bharat"
output_file = os.path.join(custom_temp_dir, "Bharat.wav")
synthesize_word(phonemes, output_file)

# Plot original and reconstructed signals
plt.figure(figsize=(10, 6))

# Original signal
plt.subplot(2, 1, 1)
plt.title('Original Signal')
audio_data_np, _ = librosa.load(audio_file, sr=source.SAMPLE_RATE)
plt.plot(np.linspace(0, len(audio_data_np) / source.SAMPLE_RATE, num=len(audio_data_np)), audio_data_np)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Reconstructed signal
reconstructed_audio_data, _ = librosa.load(output_file, sr=source.SAMPLE_RATE)
plt.subplot(2, 1, 2)
plt.title('Reconstructed Signal')
plt.plot(np.linspace(0, len(reconstructed_audio_data) / source.SAMPLE_RATE, num=len(reconstructed_audio_data)), reconstructed_audio_data)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

In [ ]:
import IPython.display as ipd 
ipd.display(ipd.Audio(r"Bharat.wav"))
